In [7]:
import os
import json

import numpy as np
from scipy.spatial.transform import Rotation as R

from Motion.BVH import load, save
from Motion.Animation import Animation
from Motion.AnimationStructure import get_kinematic_chain
from Motion.Quaternions import Quaternions
from Motion.plot_script import plot_3d_motion

from body_comparison_realtime.utils import BODY38_idx2name, BODY38_name2idx

In [8]:
fname = "BP_Mixamo_New10_Scene_1_16_0.json"

with open(fname, "r") as fp:
    data = json.load(fp)

In [9]:
rotations = []
motion_data = data['motion_data']
for key in motion_data.keys():
    if key.endswith("RX") or key.endswith("RY") or key.endswith("RZ"):
        if motion_data[key]:
            rotations.append(motion_data[key])
rotations = np.array(rotations)
rotations = rotations.T.reshape(rotations.shape[1], -1, 3)


translations = []
motion_data = data['motion_data']
for key in motion_data.keys():
    if key.endswith("TX") or key.endswith("TY") or key.endswith("TZ"):
        if motion_data[key]:
            translations.append(motion_data[key])
translations = np.array(translations)
translations = translations.T.reshape(translations.shape[1], -1, 3)


orients = Quaternions(np.array([[1, 0, 0, 0] for _ in range(rotations.shape[0])]))
offsets = np.ones((rotations.shape[0], 3))

In [10]:
new_anim = Animation(
    Quaternions.from_euler(rotations, order="xyz", world=False),
    translations,
    orients,
    offsets,
    data["parents"]
)

In [11]:
for i, name in enumerate(data["joints_names"]):
    print(i, name)

0 Hips
1 Spine
2 Spine1
3 Spine2
4 Neck
5 Head
6 HeadTop_End
7 RightShoulder
8 RightArm
9 RightForeArm
10 RightHand
11 RightHandThumb1
12 RightHandThumb2
13 RightHandThumb3
14 RightHandThumb4
15 RightHandIndex1
16 RightHandIndex2
17 RightHandIndex3
18 RightHandIndex4
19 RightHandMiddle1
20 RightHandMiddle2
21 RightHandMiddle3
22 RightHandMiddle4
23 RightHandRing1
24 RightHandRing2
25 RightHandRing3
26 RightHandRing4
27 RightHandPinky1
28 RightHandPinky2
29 RightHandPinky3
30 RightHandPinky4
31 LeftShoulder
32 LeftArm
33 LeftForeArm
34 LeftHand
35 LeftHandThumb1
36 LeftHandThumb2
37 LeftHandThumb3
38 LeftHandThumb4
39 LeftHandIndex1
40 LeftHandIndex2
41 LeftHandIndex3
42 LeftHandIndex4
43 LeftHandMiddle1
44 LeftHandMiddle2
45 LeftHandMiddle3
46 LeftHandMiddle4
47 LeftHandRing1
48 LeftHandRing2
49 LeftHandRing3
50 LeftHandRing4
51 LeftHandPinky1
52 LeftHandPinky2
53 LeftHandPinky3
54 LeftHandPinky4
55 RightUpLeg
56 RightLeg
57 RightFoot
58 RightToeBase
59 RightToe_End
60 LeftUpLeg
61 Lef

In [12]:
save(filename="test.bvh", anim=new_anim, names=data["joints_names"], frametime=1/20)

In [13]:
# Initialize array to store 3D keypoints
keypoints = []

# Process each frame
for frame_idx in range(rotations.shape[0]):
    frame_keypoints = []  # Store keypoints for all joints in the current frame

    # Process each joint
    for joint_idx in range(rotations.shape[1]):
        euler_angles = rotations[frame_idx, joint_idx]
        translation = translations[frame_idx, joint_idx]

        # Convert Euler angles to a rotation matrix
        rotation_matrix = R.from_euler('xyz', euler_angles, degrees=True).as_matrix()

        # Apply the rotation and translation to the origin (0, 0, 0)
        keypoint = rotation_matrix @ np.array([0, 0, 0]) + translation

        frame_keypoints.append(keypoint)

    # Add all joint keypoints for the current frame
    keypoints.append(frame_keypoints)

# Convert to a numpy array for easy handling
keypoints = np.array(keypoints)  # Shape: (frames, joints, 3)

In [14]:
skeleton = get_kinematic_chain(data["parents"])

In [ ]:
plot_3d_motion("test_vis.mp4", skeleton, keypoints[:200], title="Test viz", fps=20)